# Ivy Plus MARC Analysis with Enhanced Matching

This notebook processes MARC data from Ivy Plus libraries to identify unique records held by Penn that are not held by other institutions in the consortium.

## Enhanced Normalization and Matching

The matching process has been improved with specialized normalization for different fields:

1. **ISBN/LCCN Matching**: When standard identifiers are available, they are normalized and used as primary match keys
   - ISBN-10 and ISBN-13 are properly normalized to ensure consistent matching
   - LCCNs are standardized to handle different formats and prefixes

2. **Match Key Creation**: For records without standard identifiers, a composite key is created from:
   - Normalized title (with improved noise word removal)
   - Normalized edition statement
   - Normalized publication information with year extraction

3. **Match Key Validation**: Each match key is validated for quality to detect potential issues
   - Short or generic match keys are flagged
   - Match key quality metrics are saved for analysis

4. **Field Selection**: 
   - Leader (FLDR) is now included for record type identification
   - F007 is excluded as it's not appropriate for unique record identification (not all records have this field).
   - Core bibliographic fields (F001, F010, F020, F245, F250, F260) are used

This improved approach maintains the principle that different editions, printings, and formats are unique bibliographic entities while enhancing the accuracy of matching across cataloging variations.

## Initial load only - Institution-specific Processing
Converts MARC to Parquet format for faster processing, maintaining institution-specific separation. This step ensures that each institution's MARC files are converted to separate Parquet files for consistent downstream processing.

The conversion includes the leader field (FLDR) for each record while excluding the 007 field to optimize the output files. The leader contains important information about the record structure, material type, and bibliographic level.

In [1]:
import os
import pandas as pd
import pyspark
from pyspark import SparkConf
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession 
import pyspark.sql.functions as F
from pyspark.sql.types import *

# OPTIMIZED: Spark Configuration for Performance
conf = SparkConf() \
    .setAppName("PodProcessing-Optimized") \
    .setMaster("local[*]") \
    .set("spark.sql.adaptive.enabled", "true") \
    .set("spark.sql.adaptive.coalescePartitions.enabled", "true") \
    .set("spark.sql.adaptive.skewJoin.enabled", "true") \
    .set("spark.sql.adaptive.localShuffleReader.enabled", "true") \
    .set("spark.sql.shuffle.partitions", "200") \
    .set("spark.executor.memory", "8g") \
    .set("spark.driver.memory", "4g") \
    .set("spark.driver.maxResultSize", "2g") \
    .set("spark.sql.execution.arrow.pyspark.enabled", "true") \
    .set("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
    .set("spark.sql.execution.arrow.maxRecordsPerBatch", "10000") \
    .set("spark.sql.parquet.enableVectorizedReader", "true") \
    .set("spark.sql.parquet.columnarReaderBatchSize", "4096")

print("🚀 Optimized Spark Configuration:")
print("  - Adaptive Query Execution: Enabled")
print("  - Memory Allocation: Reduced and optimized")
print("  - Arrow Integration: Enabled for faster Pandas conversion")
print("  - Vectorized Reading: Enabled for Parquet files")

# Check if SparkContext already exists and stop it if so
try:
    if 'sc' in locals() or 'sc' in globals():
        print("⚠️  Existing SparkContext detected. Stopping it...")
        sc.stop()
        print("✅ Previous SparkContext stopped")
except Exception as e:
    print(f"Note: {e}")
    pass

try:
    if 'spark' in locals() or 'spark' in globals():
        print("⚠️  Existing SparkSession detected. Stopping it...")
        spark.stop()
        print("✅ Previous SparkSession stopped")
except Exception as e:
    print(f"Note: {e}")
    pass

# Initialize new SparkContext with optimized configuration
try:
    sc = SparkContext(conf=conf)
    print("✅ New SparkContext created")
except Exception as e:
    print(f"Error creating SparkContext: {e}")
    # Try getting existing context
    sc = SparkContext.getOrCreate(conf)
    print("✅ Using existing SparkContext with new configuration")

# Initialize SparkSession
spark = SparkSession(sc)

print("✅ Spark session initialized with optimized settings")
print(f"Spark UI available at: {sc.uiWebUrl}")

🚀 Optimized Spark Configuration:
  - Adaptive Query Execution: Enabled
  - Memory Allocation: Reduced and optimized
  - Arrow Integration: Enabled for faster Pandas conversion
  - Vectorized Reading: Enabled for Parquet files


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/07/15 06:35:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/07/15 06:35:03 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


✅ New SparkContext created
✅ Spark session initialized with optimized settings
Spark UI available at: http://192.168.1.37:4041


In [2]:
# Install required packages
!pip install --upgrade pip
!pip install pymarc poetry marctable fuzzywuzzy python-Levenshtein langdetect

import os
import sys

# Get the user's local bin directory for macOS
user_local_bin = os.path.expanduser('~/.local/bin')

# Add the directory to PATH if it exists
if os.path.exists(user_local_bin):
    os.environ['PATH'] += os.pathsep + user_local_bin
    print(f"Added {user_local_bin} to PATH")

# Also add Python's user site-packages bin directory
python_user_bin = os.path.join(sys.prefix, 'bin')
if os.path.exists(python_user_bin):
    os.environ['PATH'] += os.pathsep + python_user_bin
    print(f"Added {python_user_bin} to PATH")

# For Homebrew Python installations on macOS
homebrew_bin = '/usr/local/bin'
if os.path.exists(homebrew_bin) and homebrew_bin not in os.environ['PATH']:
    os.environ['PATH'] += os.pathsep + homebrew_bin
    print(f"Added {homebrew_bin} to PATH")

# Check if marctable is accessible
import shutil
if shutil.which('marctable'):
    print("✅ marctable command found in PATH")
else:
    print("⚠️  marctable command not found in PATH - checking alternative locations...")
    # Try to find marctable in common locations
    possible_locations = [
        os.path.expanduser('~/Library/Python/3.11/bin'),
        os.path.expanduser('~/Library/Python/3.10/bin'),
        os.path.expanduser('~/Library/Python/3.9/bin'),
        '/opt/homebrew/bin',
        '/usr/local/bin',
    ]
    
    for loc in possible_locations:
        marctable_path = os.path.join(loc, 'marctable')
        if os.path.exists(marctable_path):
            os.environ['PATH'] += os.pathsep + loc
            print(f"✅ Found marctable in {loc} and added to PATH")
            break

print("\n✅ All packages installed and environment configured")
print(f"Current PATH: {os.environ['PATH']}")

  Using cached pip-25.1.1-py3-none-any.whl.metadata (3.6 kB)
Using cached pip-25.1.1-py3-none-any.whl (1.8 MB)
  Attempting uninstall: pip
    Found existing installation: pip 23.3.1
    Uninstalling pip-23.3.1:
      Successfully uninstalled pip-23.3.1
  Using cached fuzzywuzzy-0.18.0-py2.py3-none-any.whl.metadata (4.9 kB)
  Using cached langdetect-1.0.9-py3-none-any.whl
  Using cached fastjsonschema-2.21.1-py3-none-any.whl.metadata (2.2 kB)
  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached anyio-4.9.0-py3-none-any.whl.metadata (4.7 kB)
  Using cached pycparser-2.22-py3-none-any.whl.metadata (943 bytes)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
Using cached fastjsonschema-2.21.1-py3-none-any.whl (23 kB)
Using cached httpx-0.28.1-py3-none-any.whl (73 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 35.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 57.6 MB/s eta 0:00:00
Using cached fuzzywuzzy-0.18.0-

In [3]:
# OPTIMIZATION: Replace Python UDFs with Spark SQL Functions

def create_match_key_spark(df):
    """
    Create match keys using pure Spark SQL functions - MUCH faster than UDFs
    """
    return df.withColumn("match_key", 
        F.concat_ws("_",
            # Normalize title using SQL functions
            F.when(F.col("F245").isNotNull(),
                F.regexp_replace(
                    F.regexp_replace(
                        F.regexp_replace(
                            F.lower(F.trim(F.col("F245"))),
                            "^(the|a|an)\\s+", ""  # Remove leading articles
                        ),
                        "[^a-z0-9\\s]", ""  # Remove special characters
                    ),
                    "\\s+", " "  # Normalize whitespace
                )
            ).otherwise(""),
            
            # ENHANCED: Normalize edition with spelled-out numbers
            F.when(F.col("F250").isNotNull(),
                # First convert spelled-out editions to numbers
                F.regexp_replace(
                    F.regexp_replace(
                        F.regexp_replace(
                            F.regexp_replace(
                                F.regexp_replace(
                                    F.regexp_replace(
                                        F.lower(F.col("F250")),
                                        "\\bfirst\\b", "1"
                                    ),
                                    "\\bsecond\\b", "2"
                                ),
                                "\\bthird\\b", "3"
                            ),
                            "\\bfourth\\b", "4"
                        ),
                        "\\bfifth\\b", "5"
                    ),
                    # Then normalize all numeric editions to "N ed" format
                    "(\\d+)(?:st|nd|rd|th)?\\s*(?:ed\\.?|edition)", "$1 ed"
                )
            ).otherwise(""),
            
            # Extract year from publication using SQL functions
            F.when(F.col("F260").isNotNull(),
                F.regexp_extract(F.col("F260"), "(1[0-9]{3}|20[0-9]{2})", 1)
            ).otherwise("")
        )
    )

def normalize_ids_spark(df):
    """
    Normalize ISBN and LCCN using Spark SQL functions
    """
    return df.withColumn("normalized_isbn",
        F.when(F.col("F020").isNotNull(),
            # Enhanced ISBN extraction - more specific pattern
            F.regexp_replace(
                F.regexp_extract(F.col("F020"), "([0-9]{9,13}[0-9Xx]?)", 1),
                "[^0-9X]", ""
            )
        )
    ).withColumn("normalized_lccn", 
        F.when(F.col("F010").isNotNull(),
            F.regexp_replace(
                F.trim(F.col("F010")),
                "[^a-zA-Z0-9\\-]", ""
            )
        )
    )

def add_id_list_spark(df):
    """
    Create id_list using Spark SQL array functions
    """
    return df.withColumn("id_list",
        F.array_remove(
            F.array(
                F.when(F.col("normalized_isbn").isNotNull() & (F.col("normalized_isbn") != ""), 
                       F.col("normalized_isbn")),
                F.when(F.col("normalized_lccn").isNotNull() & (F.col("normalized_lccn") != ""), 
                       F.col("normalized_lccn"))
            ),
            None
        )
    )

def validate_match_key_spark(df):
    """
    Validate match keys using Spark SQL functions
    """
    return df.withColumn("is_valid_match_key",
        (F.length(F.col("match_key")) >= 5) &
        (~F.col("match_key").rlike("^(book|text|edition|volume|vol|publication|report)_\\d+$"))
    ).withColumn("match_key_message",
        F.when(F.length(F.col("match_key")) < 5, "Match key too short")
         .when(F.col("match_key").rlike("^(book|text|edition|volume|vol|publication|report)_\\d+$"), "Generic match key")
         .otherwise("Valid match key")
    )

def process_institution_optimized(df, institution_name):
    """
    Apply all optimizations to an institution's DataFrame
    """
    return (df
        .withColumn("source", F.lit(institution_name))
        .transform(normalize_ids_spark)
        .transform(create_match_key_spark)
        .transform(add_id_list_spark)
        .transform(validate_match_key_spark)
    )

print("✅ Optimized Spark SQL functions loaded - ready to replace UDFs")
print("📚 Enhanced edition normalization now handles:")
print("   - Spelled-out editions (first, second, third, fourth, fifth)")
print("   - Numeric editions with suffixes (1st, 2nd, 3rd, 4th)")
print("   - Various edition formats (ed., edition)")
print("   - All normalized to consistent 'N ed' format")

✅ Optimized Spark SQL functions loaded - ready to replace UDFs
📚 Enhanced edition normalization now handles:
   - Spelled-out editions (first, second, third, fourth, fifth)
   - Numeric editions with suffixes (1st, 2nd, 3rd, 4th)
   - Various edition formats (ed., edition)
   - All normalized to consistent 'N ed' format


In [ ]:
# Institution-Specific MARC to Parquet Conversion Functions

import os
import tempfile
import glob
import logging
from typing import Optional, Dict, List, Tuple
import re
from pymarc import Record, MARCReader

# Setup logging for MARC conversion
log_dir = 'pod-processing-outputs/logs'
os.makedirs(log_dir, exist_ok=True)
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler(os.path.join(log_dir, 'marc2parquet.log')),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

def extract_institution_from_filename(filename: str) -> str:
    """Extract institution name from filename patterns"""
    base = os.path.basename(filename)
    
    # For files from pod-processing-outputs/final/ like "harvard_updates-001.mrc"
    if '_' in base:
        return base.split('_')[0]
    
    # Pattern: institution-date-descriptor-format.ext
    match = re.match(r'^([a-z]+)-[\d\-]+-.*\.mrc$', base)
    if match:
        return match.group(1)
    
    # Pattern: institution-descriptor.ext
    match = re.match(r'^([a-z]+)-.*\.mrc$', base)
    if match:
        return match.group(1)
    
    # Default: use the first word
    return base.split('-')[0].split('.')[0]

def safe_read_marc_file_with_recovery(file_path: str, temp_output: str) -> Tuple[int, Dict]:
    """Read MARC file with maximum error recovery and minimal validation"""
    total_records = 0
    valid_records = 0
    report = {"total_attempted": 0, "parsed": 0, "errors": 0}
    
    try:
        with open(file_path, 'rb') as file, open(temp_output, 'wb') as outfile:
            reader = MARCReader(file, to_unicode=True, force_utf8=True, utf8_handling='replace')
            
            for record_number, record in enumerate(reader, 1):
                total_records += 1
                
                if record is None:
                    report["errors"] += 1
                    continue
                
                try:
                    outfile.write(record.as_marc())
                    valid_records += 1
                except Exception as e:
                    report["errors"] += 1
                    logger.warning(f"Error writing record {record_number}: {str(e)}")
    
    except Exception as e:
        logger.error(f"Failed to read {file_path}: {str(e)}")
        
    report["total_attempted"] = total_records
    report["parsed"] = valid_records
    
    if total_records > 0:
        report["success_rate"] = (valid_records / total_records) * 100
    else:
        report["success_rate"] = 0
        
    return valid_records, report

def get_institution_specific_marc_files() -> List[Tuple[str, str]]:
    """Get all institution-specific MARC files from processed outputs"""
    institution_file_pairs = []
    
    # PRIMARY: Look for processed MARC files in the final output directory
    final_dir = 'pod-processing-outputs/final'
    
    if os.path.exists(final_dir):
        # Get all .mrc files from the final directory
        final_marc_files = glob.glob(os.path.join(final_dir, '*.mrc'))
        
        for file in final_marc_files:
            # Extract institution from filename (e.g., "harvard_updates-001.mrc" -> "harvard")
            institution = extract_institution_from_filename(file)
            institution_file_pairs.append((institution, file))
            
        print(f"Found {len(final_marc_files)} processed MARC files in {final_dir}")
    
    # SECONDARY: Check the export directory for the latest export package
    export_dir = 'pod-processing-outputs/export'
    if os.path.exists(export_dir) and not institution_file_pairs:
        # Find the most recent export package
        export_packages = glob.glob(os.path.join(export_dir, 'marc_export_*'))
        if export_packages:
            latest_export = sorted(export_packages)[-1]  # Get most recent by timestamp
            export_marc_files = glob.glob(os.path.join(latest_export, '*.mrc'))
            
            for file in export_marc_files:
                # Skip non-MARC files
                if file.endswith('.txt'):
                    continue
                institution = extract_institution_from_filename(file)
                institution_file_pairs.append((institution, file))
            
            print(f"Found {len(export_marc_files)} MARC files in latest export: {latest_export}")
    
    # FALLBACK: If no processed files found, check for raw files
    if not institution_file_pairs:
        print("No processed files found in pod-processing-outputs/final or export directories")
        print("Falling back to raw MARC files in pod_*/file directories")
        
        # Look for marc files in institution directories
        institution_dirs = glob.glob("pod_*/file")
        
        for institution_dir in institution_dirs:
            institution = institution_dir.split('/')[0].replace('pod_', '')
            
            # Look for .mrc files only (no XML)
            mrc_files = glob.glob(f"{institution_dir}/**/*.mrc", recursive=True)
            for file in mrc_files:
                institution_file_pairs.append((institution, file))
    
    # Remove duplicates and sort
    unique_pairs = list(set(institution_file_pairs))
    unique_pairs.sort(key=lambda x: (x[0], x[1]))
    
    print(f"\nTotal institution-specific MARC files to process: {len(unique_pairs)}")
    for institution, file in unique_pairs:
        print(f"  - {institution}: {file}")
    
    return unique_pairs

def process_file_with_recovery(file: str, institution: str) -> bool:
    """Process a MARC file with maximum error recovery"""
    try:
        # Create output directory if it doesn't exist
        os.makedirs('pod-processing-outputs', exist_ok=True)
        
        # Create a temporary file for processing
        with tempfile.NamedTemporaryFile(delete=False) as temp:
            temp_file = temp.name
        
        # Create institution-specific output filename
        base = os.path.basename(file)
        output_file = os.path.join('pod-processing-outputs', 
                                   f"{institution}_{base.replace('.mrc', '-marc21.parquet')}")
        
        # Process MARC file
        written_count, report = safe_read_marc_file_with_recovery(file, temp_file)
        
        # Proceed if we have at least some records
        if written_count == 0:
            error_msg = f"No records could be processed from {file}"
            logger.error(error_msg)
            print(f"ERROR: {error_msg}")
            return False
        
        # Run marctable command - FLDR is included by default
        marctable_cmd = f'marctable parquet {temp_file} {output_file}'
        marctable_msg = f"Running marctable: {marctable_cmd}"
        logger.info(marctable_msg)
        print(marctable_msg)
        exit_status = os.system(marctable_cmd)
        
        if exit_status != 0:
            error_msg = f"marctable command failed for {institution} file {file}"
            logger.error(error_msg)
            print(f"ERROR: {error_msg}")
            return False
        else:
            success_msg = f"SUCCESS: Created {output_file} with {written_count} {institution} records ({report.get('success_rate', 0):.1f}% success rate)"
            logger.info(success_msg)
            print(success_msg)
            print(f"  Note: FLDR (leader) field is included by default in marctable output")
            return True
            
    except Exception as e:
        error_msg = f"Unexpected error processing {institution} file {file}: {str(e)}"
        logger.error(error_msg)
        print(f"ERROR: {error_msg}")
        return False
        
    finally:
        if 'temp_file' in locals() and temp_file and os.path.exists(temp_file):
            try:
                os.remove(temp_file)
            except Exception as e:
                logger.error(f"Cleanup error for {temp_file}: {str(e)}")

def marc2parquet_institution_specific(force_reprocess=False):
    """
    Convert institution-specific MARC to Parquet with maximum error recovery
    
    Args:
        force_reprocess: If True, reprocess even if parquet files already exist
    """
    # Check if previous processing has been done
    if not os.path.exists('pod-processing-outputs/final'):
        print("WARNING: No processed files found in pod-processing-outputs/final/")
        print("Consider running ivyplus-updated-marc-pyspark.ipynb first for better results")
    
    institution_file_pairs = get_institution_specific_marc_files()
    
    if not institution_file_pairs:
        error_msg = "No institution-specific MARC files found to process"
        logger.error(error_msg)
        print(f"ERROR: {error_msg}")
        return False
    
    results = []
    institution_summary = {}
    
    for institution, file in institution_file_pairs:
        if institution not in institution_summary:
            institution_summary[institution] = {"total": 0, "success": 0, "failed": 0}
        
        institution_summary[institution]["total"] += 1
        
        # Create institution-specific output filename
        base = os.path.basename(file)
        output_file = os.path.join('pod-processing-outputs', 
                                   f"{institution}_{base.replace('.mrc', '-marc21.parquet')}")

        # Skip if already processed unless force_reprocess is True
        if not force_reprocess and os.path.exists(output_file):
            skip_msg = f"Skipping already processed {institution} file {file}"
            logger.info(skip_msg)
            print(skip_msg)
            institution_summary[institution]["success"] += 1
            results.append(True)
            continue
            
        result = process_file_with_recovery(file, institution)
        results.append(result)
        
        if result:
            institution_summary[institution]["success"] += 1
        else:
            institution_summary[institution]["failed"] += 1
    
    # Print summary by institution
    print("\n=== Institution Processing Summary ===")
    for institution, stats in institution_summary.items():
        print(f"{institution.upper()}: Processed {stats['total']} files - {stats['success']} succeeded, {stats['failed']} failed")
    
    # Overall success rate
    total_success = sum(results)
    total_files = len(results)
    if total_files > 0:
        print(f"\nOverall: Successfully processed {total_success} of {total_files} files ({total_success/total_files*100:.1f}%)")
        return total_success == total_files
    else:
        print("\nNo files were processed")
        return False

# Check if conversion is needed or if we can skip directly to processing
print("Checking for existing parquet files...")
existing_parquet = glob.glob("pod-processing-outputs/*_marc21.parquet")
if existing_parquet:
    print(f"Found {len(existing_parquet)} existing parquet files")
    print("You can skip to the next cell unless you want to reprocess")
else:
    print("No parquet files found. Running conversion...")
    marc2parquet_institution_specific()

Checking for existing parquet files...
No parquet files found. Running conversion...
Found 13 processed MARC files in pod-processing-outputs/final

Total institution-specific MARC files to process: 13
  - brown: pod-processing-outputs/final/brown_filtered.mrc
  - chicago: pod-processing-outputs/final/chicago_filtered.mrc
  - columbia: pod-processing-outputs/final/columbia_filtered.mrc
  - cornell: pod-processing-outputs/final/cornell_filtered.mrc
  - dartmouth: pod-processing-outputs/final/dartmouth_filtered.mrc
  - duke: pod-processing-outputs/final/duke_filtered.mrc
  - harvard: pod-processing-outputs/final/harvard_filtered.mrc
  - johns: pod-processing-outputs/final/johns_filtered.mrc
  - mit: pod-processing-outputs/final/mit_filtered.mrc
  - penn: pod-processing-outputs/final/penn_filtered.mrc
  - princeton: pod-processing-outputs/final/princeton_filtered.mrc
  - stanford: pod-processing-outputs/final/stanford_filtered.mrc
  - yale: pod-processing-outputs/final/yale_filtered.mrc
Ru

In [ ]:
# Main Processing with Optimized Functions
import pyspark.sql.functions as F
from pyspark.sql.functions import col, explode, size, array_contains, collect_set
from functools import reduce

# Find all institution parquet files in pod-processing-outputs directory
import glob
import os

parquet_files = glob.glob("pod-processing-outputs/*_marc21.parquet")

print(f"Found {len(parquet_files)} institution-specific parquet files to process")

# Check if we have any files to process
if not parquet_files:
    print("ERROR: No parquet files found!")
    print("Please run the previous cell to convert MARC files to Parquet format.")
    raise ValueError("No parquet files found in pod-processing-outputs/")

print("Files to process:")
for f in parquet_files:
    print(f"  - {f}")

# Process each institution's data with optimized functions
all_dfs = []
total_records_by_institution = {}

for parquet_file in parquet_files:
    # Extract institution name from filename
    filename = os.path.basename(parquet_file)
    institution = filename.split('_')[0]
    
    print(f"\nProcessing {institution} from {parquet_file}")
    
    # Read and process the parquet file
    df = spark.read.parquet(parquet_file)
    processed_df = process_institution_optimized(df, institution)
    
    # Get statistics without caching individual DataFrames
    total_records = processed_df.count()
    total_records_by_institution[institution] = total_records
    
    if total_records > 0:
        # Select only needed columns and add to list
        selected_df = processed_df.select(
            "F001", "source", "match_key", "id_list", 
            "is_valid_match_key", "match_key_message"
        )
        all_dfs.append(selected_df)
        
        # Quick validation check (sample-based for efficiency)
        sample_validation = processed_df.select("is_valid_match_key").sample(
            fraction=min(10000/total_records, 1.0), seed=42
        ).groupBy("is_valid_match_key").count().collect()
        
        valid_sample = next((r["count"] for r in sample_validation if r["is_valid_match_key"]), 0)
        total_sample = sum(r["count"] for r in sample_validation)
        
        if total_sample > 0:
            est_valid_pct = (valid_sample / total_sample) * 100
            print(f"  - {total_records:,} records processed")
            print(f"  - Estimated {est_valid_pct:.1f}% valid match keys (based on sample)")
    else:
        print(f"  - WARNING: No records found in {parquet_file}")

# Check if we have any DataFrames to union
if not all_dfs:
    print("ERROR: No DataFrames to combine!")
    raise ValueError("No valid institution data found to process")

# Efficient union of all DataFrames
print("\nCombining all institution data into a single DataFrame...")
all_df = reduce(lambda df1, df2: df1.union(df2), all_dfs)

# Cache the combined DataFrame (this is where caching makes most sense)
all_df.cache()

# Force computation and get actual count
print("Materializing combined DataFrame...")
total_keys = all_df.count()
print(f"Total records across all institutions: {total_keys:,}")

# Define the matching key array - COMBINED APPROACH
print("\n🔄 Using COMBINED matching approach: ISBN/LCCN AND match_key")
all_df = all_df.withColumn("key_array",
    F.array_distinct(
        F.concat(
            F.col("id_list"),  # ISBN/LCCN identifiers
            F.array(F.col("match_key"))  # Always include match_key
        )
    )
)

# Efficient validation counting in a single pass
if total_keys > 0:
    print("\nAnalyzing match key quality...")
    validation_stats = all_df.groupBy("is_valid_match_key").count().collect()
    stats_dict = {row["is_valid_match_key"]: row["count"] for row in validation_stats}
    
    valid_keys = stats_dict.get(True, 0)
    invalid_keys = stats_dict.get(False, 0)
    
    print(f"\nMatch key validation results:")
    print(f"  • Valid match keys: {valid_keys:,} ({valid_keys/total_keys*100:.1f}%)")
    print(f"  • Invalid match keys: {invalid_keys:,} ({invalid_keys/total_keys*100:.1f}%)")
    
    if invalid_keys > 0:
        print("\nInvalid match key reasons:")
        all_df.filter(col("is_valid_match_key") == False) \
            .groupBy("match_key_message") \
            .count() \
            .orderBy(col("count").desc()) \
            .show(10, False)

# Explode the key_array
all_df_exploded = all_df.withColumn("key", explode("key_array"))

# Print key array statistics
print("\n📊 Key array statistics:")
key_array_sizes = all_df.select(
    F.size("key_array").alias("num_keys")
).groupBy("num_keys").count().orderBy("num_keys")
print("Distribution of number of keys per record:")
key_array_sizes.show()

# Unpersist the cached combined DataFrame to free memory
all_df.unpersist()

print(f"\n✅ Processing complete. Ready for uniqueness analysis.")

In [ ]:
# Uniqueness Analysis and Overlap Detection
from pyspark.sql.functions import collect_set, array_contains, size, col

# Group by key and collect sources where that key appears
grouped = all_df_exploded.groupBy("key").agg(
    collect_set("source").alias("sources"),
    F.count("*").alias("record_count")
)

# Broadcast the grouped DataFrame for more efficient joins
grouped_broadcast = F.broadcast(grouped)

# Find Penn records that exist in OTHER libraries
# A Penn record is NOT unique if it exists in ANY other library
penn_keys_in_other_libs = grouped_broadcast.filter(
    (array_contains(col("sources"), "penn")) & 
    (F.size(col("sources")) > 1)  # Penn + at least one other library
).select("key")

# Get Penn records that are truly unique to Penn
# Start with all Penn records
all_penn_exploded = all_df_exploded.filter(col("source") == "penn")

# Anti-join to remove Penn records found in other libraries
unique_penn_exploded = all_penn_exploded.join(
    penn_keys_in_other_libs,  # No need to re-broadcast
    on="key", 
    how="left_anti"
)

# Deduplicate by Penn's F001 (not match_key) to get unique Penn records
unique_penn = unique_penn_exploded.drop("key").dropDuplicates(["F001"])

# Cache the unique Penn records for better performance
unique_penn.cache()

# Calculate statistics efficiently
unique_penn_count = unique_penn.count()  # Force cache materialization

# Get total Penn records from the deduplicated exploded DataFrame
total_penn = all_penn_exploded.select("F001").distinct().count()

print(f"\n=== Analysis Results ===")
print(f"Total Penn records: {total_penn:,}")
print(f"Unique Penn records: {unique_penn_count:,}")

# Add robust checking for division by zero
if total_penn > 0:
    print(f"Uniqueness rate: {unique_penn_count/total_penn*100:.1f}%")
    print(f"Overlap rate: {(total_penn - unique_penn_count)/total_penn*100:.1f}%")
else:
    print("Uniqueness rate: N/A (no Penn records found)")

# For analysis, let's also see overlap statistics
print("\n=== Overlap Analysis ===")

# More efficient: get Penn overlap stats without re-filtering
penn_keys = grouped.filter(array_contains(col("sources"), "penn")).cache()

penn_overlap_stats = penn_keys \
    .withColumn("num_libraries", F.size(col("sources"))) \
    .groupBy("num_libraries").count() \
    .orderBy("num_libraries")

print("Distribution of Penn records by number of libraries holding them:")
penn_overlap_stats.show()

# Save results with consistent paths using pod-processing-outputs directory
output_dir = "pod-processing-outputs"

# Save unique Penn records
unique_penn.write.mode("overwrite").parquet(f"{output_dir}/unique_penn.parquet")

# Save detailed overlap information for analysis
# Note: Using cached penn_keys for efficiency
penn_with_overlap_info = all_penn_exploded.join(
    penn_keys.select("key", "sources", F.size("sources").alias("num_libraries")),
    on="key",
    how="left"
).drop("key")

penn_with_overlap_info.write.mode("overwrite").parquet(f"{output_dir}/penn_overlap_analysis.parquet")

# Save validation statistics for analysis
validation_stats = all_df.select("F001", "match_key", "is_valid_match_key", "match_key_message", "id_list") \
    .filter(col("source") == "penn")

validation_stats.write.mode("overwrite").parquet(f"{output_dir}/match_key_validation_stats.parquet")

# Unpersist cached DataFrames to free memory
penn_keys.unpersist()
unique_penn.unpersist()

In [ ]:
# Data Source Validation (Updated: July 2025)
# Validates Penn MARC data sources and ensures current data is used
# Requires explicit confirmation for legacy data usage

# Use Leader field FLDR to make a print set from unique penn and non-print
from pyspark.sql.functions import col, substring, when, concat, lit
import pyspark.sql.functions as F
import glob
import os
import re
from datetime import datetime

# Add this after the imports
if 'output_dir' not in locals():
    output_dir = "pod-processing-outputs"

# Load the unique Penn dataset if not already loaded
if 'unique_penn' not in locals() or unique_penn is None:
    print("Loading unique Penn dataset...")
    unique_penn = spark.read.parquet(f"{output_dir}/unique_penn.parquet")
else:
    print("Using existing unique_penn DataFrame")

# CRITICAL: Verify Penn data currency before processing
def verify_penn_data_source(matching_files):
    """
    Verify the Penn data source and warn if using outdated data
    """
    if not matching_files:
        return None
        
    selected_file = matching_files[0]
    file_info = {
        'path': selected_file,
        'filename': os.path.basename(selected_file),
        'is_legacy': 'penn-2022-07-20' in selected_file,
        'is_processed': 'pod-processing-outputs' in selected_file
    }
    
    # Extract date from filename if possible
    date_pattern = r'(\d{4}-\d{2}-\d{2})'
    date_match = re.search(date_pattern, file_info['filename'])
    if date_match:
        file_info['data_date'] = date_match.group(1)
    else:
        file_info['data_date'] = 'unknown'
    
    return file_info

# Load full Penn records - prioritize most recent processed data
penn_full_paths = [
    # PRIMARY: Penn parquet files from current processing pipeline
    "pod-processing-outputs/penn_*updates*marc21.parquet",
    
    # SECONDARY: Any Penn parquet files in processing outputs
    "pod-processing-outputs/penn_*.parquet",
    
    # TERTIARY: Check for raw Penn parquet files (less preferred)
    "pod_penn/file/**/*.parquet"
    
    # REMOVED: Legacy path - DO NOT USE unless absolutely necessary
    # "/home/jovyan/work/marc/parquet/penn-2022-07-20-full-marc21.parquet"
]

# Add data source verification
penn_full = None
selected_source = None

print("\n=== PENN DATA SOURCE VERIFICATION ===")
for path_pattern in penn_full_paths:
    try:
        matching_files = glob.glob(path_pattern, recursive=True)
        if matching_files:
            # Sort files by modification time to get most recent
            matching_files.sort(key=lambda x: os.path.getmtime(x), reverse=True)
            
            source_info = verify_penn_data_source(matching_files)
            if source_info:
                print(f"\nFound Penn records at: {source_info['path']}")
                print(f"  - Source type: {'Processed updates' if source_info['is_processed'] else 'Raw data'}")
                print(f"  - Data date: {source_info['data_date']}")
                
                # Warn if data appears old
                if source_info['data_date'] != 'unknown':
                    try:
                        data_date = datetime.strptime(source_info['data_date'], '%Y-%m-%d')
                        days_old = (datetime.now() - data_date).days
                        if days_old > 365:
                            print(f"  ⚠️  WARNING: Data is {days_old} days old!")
                            print(f"  ⚠️  Results may not reflect current Penn holdings")
                    except:
                        pass
                
                # Load the data
                penn_full = spark.read.parquet(source_info['path'])
                selected_source = source_info
                
                # Verify record count and sample for currency check
                record_count = penn_full.count()
                print(f"  - Total records: {record_count:,}")
                
                # Sample check for recent cataloging activity
                if 'F005' in penn_full.columns:
                    recent_updates = penn_full.filter(
                        col("F005").rlike("202[4-5]")
                    ).count()
                    recent_percentage = (recent_updates / record_count * 100) if record_count > 0 else 0
                    print(f"  - Recently updated records (2024-2025): {recent_updates:,} ({recent_percentage:.1f}%)")
                    
                    if recent_percentage < 5:
                        print(f"  ⚠️  WARNING: Only {recent_percentage:.1f}% of records updated recently")
                        print(f"  ⚠️  Data may be significantly outdated")
                
                break
    except Exception as e:
        print(f"Error checking {path_pattern}: {str(e)}")
        continue

# Final fallback with strong warning
if penn_full is None:
    print("\n⚠️  CRITICAL WARNING: No current Penn data found!")
    print("As a last resort, checking for legacy data...")
    
    legacy_path = "/home/jovyan/work/marc/parquet/penn-2022-07-20-full-marc21.parquet"
    if os.path.exists(legacy_path):
        response = input("\n🚨 Found 2022 Penn data. This is SEVERELY OUTDATED. Use anyway? (yes/no): ")
        if response.lower() == 'yes':
            penn_full = spark.read.parquet(legacy_path)
            selected_source = {'is_legacy': True, 'filename': 'penn-2022-07-20-full-marc21.parquet'}
            print("⚠️  Using 2022 data - results will NOT reflect current Penn holdings!")
        else:
            raise FileNotFoundError("No Penn full MARC records found and user declined legacy data")
    else:
        print("ERROR: Could not find Penn full MARC records!")
        print("Please ensure Penn MARC data has been converted to Parquet format.")
        print("Run the previous cells to process MARC files first.")
        raise FileNotFoundError("Penn full MARC records not found")

print("\n=== PROCEEDING WITH ANALYSIS ===")
if selected_source and selected_source.get('is_legacy'):
    print("⚠️  USING OUTDATED DATA - RESULTS MAY BE INACCURATE")

# OPTIMIZATION: Use join instead of SQL IN clause for better performance
unique_penn_ids = unique_penn.select("F001").distinct()
unique_penn_full = penn_full.join(unique_penn_ids, on="F001", how="inner")

# OPTIMIZATION: Apply all transformations in a single chain
unique_penn_with_material_type = (unique_penn_full
    # Filter out 533 fields first
    .filter(col("F533").isNull())
    # Add material type columns
    .withColumn("record_type", substring(col("FLDR"), 7, 1))
    .withColumn("bib_level", substring(col("FLDR"), 8, 1))
    .withColumn("combined_type", concat(col("record_type"), col("bib_level")))
    .withColumn("material_category", 
        when((col("record_type") == "a") & (col("bib_level").isin("m")), "print_book")
        .when((col("record_type") == "a") & (col("bib_level").isin("s")), "print_serial")
        .when((col("record_type") == "c") & (col("bib_level").isin("m", "s")), "print_music")
        .when((col("record_type") == "e") & (col("bib_level").isin("m", "s")), "print_maps")
        .when(col("record_type") == "m", "electronic_resource")
        .when(col("record_type").isin("g", "k"), "visual_material")
        .when(col("record_type") == "i", "audio_material")
        .otherwise("other")
    )
    .withColumn("is_print", 
        col("material_category").isin("print_book", "print_serial", "print_music", "print_maps")
    )
)

# Cache before multiple operations
unique_penn_with_material_type.cache()

# OPTIMIZATION: Get all statistics in one pass
print("\n=== Material Type Distribution ===")
material_stats = unique_penn_with_material_type.groupBy("material_category", "is_print").count().collect()

# Process statistics
material_counts_dict = {}
print_count = 0
non_print_count = 0

for row in material_stats:
    material_counts_dict[row["material_category"]] = row["count"]
    if row["is_print"]:
        print_count += row["count"]
    else:
        non_print_count += row["count"]

total_unique = print_count + non_print_count

# Display material distribution
for category, count in sorted(material_counts_dict.items(), key=lambda x: x[1], reverse=True):
    print(f"{category}: {count:,}")

# Filter for print materials only
print_only_df = unique_penn_with_material_type.filter(col("is_print") == True)

# Add metadata if we have source information
if selected_source:
    print_only_df_with_metadata = print_only_df.withColumn(
        "processing_date", lit(datetime.now().strftime("%Y-%m-%d"))
    ).withColumn(
        "source_file", lit(selected_source.get('filename', 'unknown'))
    ).withColumn(
        "data_currency_warning", 
        lit("OUTDATED - 2022 data" if selected_source.get('is_legacy') else "Current")
    )
else:
    print_only_df_with_metadata = print_only_df

# Save datasets
unique_penn_with_material_type.write.mode("overwrite").parquet(f"{output_dir}/unique_penn_full_no_533.parquet")
print_only_df_with_metadata.write.mode("overwrite").parquet(f"{output_dir}/physical_books_no_533.parquet")

# Print final statistics
print(f"\n=== Print Material Analysis ===")
print(f"Total unique Penn records: {total_unique:,}")

if total_unique > 0:
    print(f"Print materials: {print_count:,} ({print_count/total_unique*100:.1f}%)")
    print(f"Non-print materials: {non_print_count:,} ({non_print_count/total_unique*100:.1f}%)")
    
    # Show print categories breakdown
    print("\n=== Print Material Categories ===")
    print_categories = ["print_book", "print_serial", "print_music", "print_maps"]
    for category in print_categories:
        if category in material_counts_dict:
            count = material_counts_dict[category]
            print(f"{category}: {count:,} ({count/print_count*100:.1f}% of print materials)")
else:
    print("No unique Penn records found to analyze")

# Unpersist cached DataFrame
unique_penn_with_material_type.unpersist()

# Final warning if using outdated data
if selected_source and selected_source.get('is_legacy'):
    print("\n" + "="*60)
    print("🚨 CRITICAL WARNING: Analysis completed using 2022 Penn data")
    print("🚨 Results do NOT reflect current Penn holdings")
    print("🚨 Recommended: Re-run with current Penn MARC export")
    print("="*60)

In [ ]:
# Stratified Sampling and Final Analysis
from pyspark.sql.functions import rand, col
import json
from datetime import datetime 

# Define output directory if not already defined
if 'output_dir' not in locals():
    output_dir = "pod-processing-outputs"

# Load print materials dataset if not already loaded
if 'print_only_df' not in locals() or print_only_df is None:
    print("Loading print materials dataset...")
    print_only_df_raw = spark.read.parquet(f"{output_dir}/physical_books_no_533.parquet")
    
    # Check if metadata columns exist and drop them for sampling
    metadata_cols = ["processing_date", "source_file", "data_currency_warning"]
    existing_metadata_cols = [col for col in metadata_cols if col in print_only_df_raw.columns]
    
    if existing_metadata_cols:
        print(f"Dropping metadata columns: {existing_metadata_cols}")
        print_only_df = print_only_df_raw.drop(*existing_metadata_cols)
    else:
        print_only_df = print_only_df_raw
else:
    print("Using existing print_only_df DataFrame")

# Load or compute necessary statistics if not available
if 'total_penn' not in locals() or 'unique_penn_count' not in locals():
    print("Loading required statistics...")
    # Load from saved parquet files
    if 'unique_penn' not in locals():
        unique_penn = spark.read.parquet(f"{output_dir}/unique_penn.parquet")
    unique_penn_count = unique_penn.count()
    
    # Load Penn overlap analysis to get total Penn records
    penn_overlap = spark.read.parquet(f"{output_dir}/penn_overlap_analysis.parquet")
    total_penn = penn_overlap.select("F001").distinct().count()

# Compute print statistics if not available
if 'print_count' not in locals() or 'material_counts_dict' not in locals():
    print("Computing material type statistics...")
    # Check for material_category column
    if 'material_category' not in print_only_df.columns:
        print("ERROR: material_category column not found in print_only_df")
        raise ValueError("Missing required column: material_category")
    
    material_stats = print_only_df.groupBy("material_category").count().collect()
    material_counts_dict = {row["material_category"]: row["count"] for row in material_stats}
    print_count = sum(material_counts_dict.values())

# Define sampling function with improved stratification
def create_stratified_sample(df, strata_column, sample_size=1000):
    """
    Create a stratified sample with improved randomization.
    Uses multiple passes to ensure representation of all strata.
    """
    print(f"Creating stratified sample based on {strata_column}...")
    
    # Verify the strata column exists
    if strata_column not in df.columns:
        print(f"ERROR: Column '{strata_column}' not found in DataFrame")
        print(f"Available columns: {df.columns}")
        raise ValueError(f"Missing required column: {strata_column}")
    
    # Get counts by strata for weighting
    strata_counts = df.groupBy(strata_column).count().collect()
    total_records = df.count()
    
    if total_records == 0:
        print("WARNING: No records to sample from!")
        return df
    
    strata_map = {row[strata_column]: row["count"] for row in strata_counts}
    print(f"Strata distribution:")
    for strata, count in sorted(strata_map.items()):
        print(f"  - {strata}: {count:,} records ({count/total_records*100:.2f}%)")
    
    # Calculate proportional sample sizes with minimum threshold
    min_per_strata = 5  # Ensure at least a few records from each stratum
    sample_fractions = {}
    
    for strata, count in strata_map.items():
        # Proportional sampling with minimum threshold
        if count > 0:
            # Calculate proportional share but ensure at least min_per_strata
            prop_size = max(
                min_per_strata,
                int((count / total_records) * sample_size)
            )
            
            # Don't sample more than we have
            prop_size = min(prop_size, count)
            
            # Calculate fraction
            sample_fractions[strata] = prop_size / count
    
    # First pass: Stratified sampling
    sampled_df = df.sampleBy(strata_column, fractions=sample_fractions, seed=42)
    
    # Check if we need a second pass to reach target size
    current_size = sampled_df.count()
    print(f"First pass sample size: {current_size}")
    
    if current_size < sample_size and current_size < total_records:
        # Second pass: Sample from under-represented strata
        remaining = min(sample_size - current_size, total_records - current_size)
        print(f"Need {remaining} more records to reach target sample size")
        
        # Get records not in first sample
        sampled_ids = sampled_df.select("F001").distinct()
        remaining_df = df.join(sampled_ids, on="F001", how="left_anti")
        
        remaining_count = remaining_df.count()
        if remaining_count > 0:
            # Simple random sample from remaining records
            additional_sample = remaining_df.orderBy(rand(seed=43)).limit(remaining)
            
            # Union the samples
            sampled_df = sampled_df.union(additional_sample)
            print(f"Added {min(remaining, remaining_count)} additional records")
    
    final_size = sampled_df.count()
    print(f"Final sample size: {final_size}")
    
    # Check distribution in final sample
    sample_distribution = sampled_df.groupBy(strata_column).count().collect()
    print(f"\nSample distribution by {strata_column}:")
    sample_dict = {row[strata_column]: row["count"] for row in sample_distribution}
    
    for strata_val in sorted(strata_map.keys()):
        original_count = strata_map.get(strata_val, 0)
        sample_count = sample_dict.get(strata_val, 0)
        if original_count > 0 and final_size > 0:
            print(f"  - {strata_val}: {sample_count} ({sample_count/final_size*100:.2f}% of sample vs {original_count/total_records*100:.2f}% of population)")
    
    return sampled_df

# Create a stratified sample by material category
sample_df = create_stratified_sample(print_only_df, "material_category", sample_size=1000)

# Cache the sample for better performance
sample_df.cache()

# Save the sample for API validation
sample_df.write.mode("overwrite").parquet(f"{output_dir}/statistical_sample_for_api_no_hsp.parquet")

# Convert to CSV for easier human review
# Select key fields for the CSV
sample_for_csv = sample_df.select(
    "F001", "F020", "F010", "F245", "F250", "F260", "material_category"
)

# Save as CSV (single file for easier review)
sample_for_csv.coalesce(1).write.mode("overwrite").option("header", "true").csv(f"{output_dir}/statistical_sample_for_api_no_hsp.csv")

# Generate final summary statistics in JSON format
summary_stats = {
    "processing_timestamp": datetime.now().isoformat(),
    "total_penn_records": int(total_penn),
    "unique_penn_records": int(unique_penn_count),
    "uniqueness_rate": float(unique_penn_count/total_penn) if total_penn > 0 else 0.0,
    "print_materials": int(print_count),
    "print_materials_percentage": float(print_count/unique_penn_count) if unique_penn_count > 0 else 0.0,
    "sample_size": int(sample_df.count()),
    "material_categories": {}
}

# Add material categories to summary
for category, count in sorted(material_counts_dict.items()):
    summary_stats["material_categories"][category] = {
        "count": int(count),
        "percentage": float(count/print_count*100) if print_count > 0 else 0.0
    }

# Write summary to JSON file
with open(f"{output_dir}/sample_summary_no_hsp.json", "w") as f:
    json.dump(summary_stats, f, indent=2)

# Unpersist the cached sample
sample_df.unpersist()

print("\n✅ Processing complete!")
print(f"Results saved to {output_dir}/")
print("\nFinal outputs:")
print(f"  - unique_penn.parquet: All unique Penn records")
print(f"  - physical_books_no_533.parquet: Unique Penn physical books")
print(f"  - statistical_sample_for_api_no_hsp.parquet: Statistical sample for validation")
print(f"  - statistical_sample_for_api_no_hsp.csv: CSV version of sample")
print(f"  - sample_summary_no_hsp.json: Summary statistics")

# Display summary statistics
print("\n📊 Summary Statistics:")
print(f"  - Total Penn records: {summary_stats['total_penn_records']:,}")
print(f"  - Unique Penn records: {summary_stats['unique_penn_records']:,}")
print(f"  - Uniqueness rate: {summary_stats['uniqueness_rate']*100:.1f}%")
print(f"  - Print materials: {summary_stats['print_materials']:,}")
print(f"  - Print materials percentage: {summary_stats['print_materials_percentage']:.1f}%")
print(f"  - Sample size: {summary_stats['sample_size']:,}")

# Display material category breakdown
if material_counts_dict:
    print("\n📚 Material Category Breakdown:")
    for category, info in sorted(summary_stats["material_categories"].items()):
        print(f"  - {category}: {info['count']:,} ({info['percentage']:.1f}%)")

In [ ]:
# Cleanup Cell - Run this to free all resources
def cleanup_spark_resources():
    """Clean up all cached DataFrames and temporary views"""
    try:
        # Get all cached DataFrames
        for (id, rdd) in spark.sparkContext._jsc.getPersistentRDDs().items():
            rdd.unpersist()
        
        # Drop all temporary views
        for view in spark.catalog.listTables():
            if view.isTemporary:
                spark.catalog.dropTempView(view.name)
        
        print("✅ All Spark resources cleaned up")
    except Exception as e:
        print(f"⚠️ Cleanup warning: {e}")

cleanup_spark_resources()